# CREACION DE DATASETS POR ESTACIONE

In [ ]:
import re
import pandas as pd
from pathlib import Path
import glob
import os

def _remove_accents(text):
    text = str(text)
    replacements = {
        "á":"a", "Á":"A", "é":"e", "É":"E", "í":"i", "Í":"I",
        "ó":"o", "Ó":"O", "ú":"u", "Ú":"U", "ñ":"n", "Ñ":"N"
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text

def _normalize(colname):
    s = _remove_accents(colname).lower()
    s = re.sub(r'[^a-z0-9]', '', s)
    return s

def ensure_fecha_hora(df):
    df = df.copy()
    cols = list(df.columns)
    norm = {col: re.sub(r'\s+', ' ', str(col)).strip().upper() for col in cols}

    combined_col = None
    for orig, up in norm.items():
        if 'FECHA' in up and 'HORA' in up:
            combined_col = orig
            break

    fecha_col = None
    hora_col = None
    for orig, up in norm.items():
        if up == 'FECHA' or up.startswith('FECHA ') or up.startswith('FECHA_') or up == 'DATE':
            fecha_col = orig
        if up == 'HORA' or up.startswith('HORA ') or up.startswith('HORA_') or up in ('TIME', 'HOUR'):
            hora_col = orig

    if fecha_col and hora_col:
        df = df.rename(columns={fecha_col: 'FECHA', hora_col: 'HORA'})
        return df

    if combined_col:
        combined = df[combined_col].astype(str).str.strip()
        parts = combined.str.split(r'[\sT,;]+', n=1, expand=True)
        df['FECHA'] = parts[0]
        df['HORA'] = parts[1].fillna('')
        df = df.drop(columns=[combined_col])
        return df

    date_regex = re.compile(r'\d{2}/\d{2}/\d{4}')
    time_regex = re.compile(r'^\s*\d{1,2}(:\d{2})?\s*$')

    detected_fecha = None
    for col in cols:
        sample = df[col].astype(str).dropna().head(30)
        if sample.apply(lambda x: bool(date_regex.search(x))).sum() > 0:
            if sample.apply(lambda x: bool(re.search(r'\d{2}/\d{2}/\d{4}.*\d{1,2}', x))).sum() > 0:
                parts = df[col].astype(str).str.strip().str.split(r'[\sT,;]+', n=1, expand=True)
                df['FECHA'] = parts[0]
                df['HORA'] = parts[1].fillna('')
                df = df.drop(columns=[col])
                return df
            detected_fecha = col
            break

    detected_hora = None
    for col in cols:
        sample = df[col].astype(str).dropna().head(30)
        if sample.apply(lambda x: bool(time_regex.search(x))).sum() > 0:
            detected_hora = col
            break

    if detected_fecha and detected_hora:
        df = df.rename(columns={detected_fecha: 'FECHA', detected_hora: 'HORA'})
        return df

    if detected_fecha and not detected_hora:
        parts = df[detected_fecha].astype(str).str.strip().str.split(r'[\sT,;]+', n=1, expand=True)
        df['FECHA'] = parts[0]
        df['HORA'] = parts[1].fillna('')
        if detected_fecha != 'FECHA':
            df = df.drop(columns=[detected_fecha])
        return df

    for col in cols:
        parsed = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
        if parsed.notna().sum() > 0:
            df['FECHA'] = parsed.dt.strftime('%d/%m/%Y')
            df['HORA'] = parsed.dt.strftime('%H:%M').where(parsed.notna(), pd.NA).astype(str)
            if col != 'FECHA':
                df = df.drop(columns=[col])
            return df

    df['FECHA'] = pd.NA
    df['HORA'] = pd.NA
    return df

def process_single_year_file(file_path):
    try:
        df = pd.read_csv(
            file_path,
            sep="\t",
            encoding="latin-1",
            skiprows=3,
            na_values=["", "NA", "nan"],
            skipinitialspace=True,
            dtype=str
        )
    except Exception as e:
        print(f"Error lectura archivo {file_path}:", e)
        return None

    df = ensure_fecha_hora(df)
    df.columns = df.columns.str.strip()

    if "FECHA" not in df.columns:
        print(f"Advertencia: FECHA no encontrada en {file_path}. Se omite fichero.")
        return None

    mask_fecha_valida = df["FECHA"].astype(str).str.match(r"\d{2}/\d{2}/\d{4}")
    df = df[mask_fecha_valida].copy()
    if df.empty:
        return None

    df["HORA"] = df["HORA"].astype(str).str.zfill(2).str.replace(r"\.0+$", "", regex=True)
    fecha_hora = df["FECHA"].astype(str).str.strip() + " " + df["HORA"].astype(str).str.strip() + ":00"
    df["datetime"] = pd.to_datetime(fecha_hora, dayfirst=True, errors="coerce")
    df = df[~df["datetime"].isna()].copy()
    if df.empty:
        return None

    df = df.drop(columns=["FECHA", "HORA"], errors="ignore")

    for col in df.columns:
        if col == "datetime":
            continue
        df[col] = df[col].replace(["", " "], pd.NA)
        df[col] = df[col].astype(str).str.replace(",", ".", regex=False)
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.sort_values("datetime").reset_index(drop=True)
    return df

def process_folder_and_save_csv(folder_path,
                                years=range(2004, 2026),
                                filename_prefix="df_",
                                output_dir=None,
                                pattern_base="MHEST*{}*.txt"):
    folder = Path(folder_path)
    if not folder.exists():
        print(f"No existe la carpeta {folder_path}. Se omite.")
        return None

    if output_dir is None:
        output_dir = Path.home() / "Desktop"
    else:
        output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    dfs = []
    for year in years:
        pattern = pattern_base.format(year)
        found = list(folder.glob(pattern))
        if not found:
            found = list(folder.glob(f"*{year}.txt"))

        if not found:
            print(f"Aviso: no se encontró archivo para el año {year} en {folder_path}")
            continue

        for fp in found:
            dfp = process_single_year_file(fp)
            if dfp is not None:
                dfp["year_source"] = int(year)
                dfs.append(dfp)

    if not dfs:
        print(f"No se han procesado datos válidos en {folder_path}.")
        return None

    df_complete = pd.concat(dfs, ignore_index=True, sort=False)
    df_complete = df_complete.sort_values("datetime", ascending=False).reset_index(drop=True)

    target_cols = ["datetime", "NO", "NO2", "NOx", "O3", "Veloc.", "Direc.", "Temp.", "R.Sol.", "Pres.", "year_source"]
    cols_present = [c for c in target_cols if c in df_complete.columns]
    df_complete = df_complete[cols_present]

    safe_name = re.sub(r'[^a-zA-Z0-9_]', '_', folder.name)
    out_file = output_dir / f"{filename_prefix}_{safe_name}_completo.csv"

    df_complete.to_csv(out_file, index=False, encoding="utf-8")
    print(f"CSV guardado en: {out_file}  Filas: {len(df_complete)} Columnas: {len(df_complete.columns)}")
    return out_file

# Lista de carpetas a procesar
list_of_folders = [
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Pla",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Rabassa",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 2/Elxe/Elxe_agroalimentari",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 2/Elxe/Elxe_bomberos",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 3/Gandia",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 3/Beniganim",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 3/Ontinyent",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Av_Francia",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_B_Sur",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Burjasot",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Moli",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Politecnic",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Quart_Poblet",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Silla",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Valencia/V_Vivers",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Caudete_Fuentes",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 4/Buñol",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 5/Paterna",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 5/Torrebaja",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 5/Villamarxant",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 5/Villar_Arzobispo",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Sagunto/S_CEA",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Sagunto/S_Nord",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Sagunto/S_Port",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Albalat_Taronjers",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Algar_Palancia",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 6/Viver",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Burriana",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Almassora",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Ermita",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Grau",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Patronat",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Penyeta",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Cirat",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Onda",

    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 8/Alcora",
    
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 9/Coratxa",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 9/Morella",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 9/Sant_Jordi",
    "/Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 9/Zorita"
    
    # añade aquí más rutas a carpetas con los txt si las hay
]

# Salvar en el Escritorio del usuario
output_folder = Path.home() / "Desktop"

# Procesar cada carpeta y generar un CSV por carpeta, años 2006 a 2025
for folder in list_of_folders:
    process_folder_and_save_csv(folder_path=folder, years=range(2004, 2026), output_dir=output_folder)


Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
Aviso: no se encontró archivo para el año 2006 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
Aviso: no se encontró archivo para el año 2007 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
Aviso: no se encontró archivo para el año 2008 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
Aviso: no se encontró archivo para el año 2009 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Alacant/Alacant_Babel
CSV guardado en: /Users/benjamincarbonell/Desktop/df__Alacant_Babel_completo.csv  Filas: 134778 Columnas: 10
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transect

/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(["", " "], pd.NA)


CSV guardado en: /Users/benjamincarbonell/Desktop/df__Alacant_Rabassa_completo.csv  Filas: 138279 Columnas: 11
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos
Aviso: no se encontró archivo para el año 2006 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos
Aviso: no se encontró archivo para el año 2007 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos
Aviso: no se encontró archivo para el año 2008 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos
Aviso: no se encontró archivo para el año 2009 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/El_Pinos


/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(["", " "], pd.NA)
/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(["", " "], pd.NA)


CSV guardado en: /Users/benjamincarbonell/Desktop/df__El_Pinos_completo.csv  Filas: 133238 Columnas: 11
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
Aviso: no se encontró archivo para el año 2006 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
Aviso: no se encontró archivo para el año 2007 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
Aviso: no se encontró archivo para el año 2008 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
Aviso: no se encontró archivo para el año 2009 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 1/Elda
CSV guardado en: /Users/benjamincarbonell/Desktop/df__Elda_completo.csv  Filas: 133989 Columnas: 7
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 2/Elxe

...

CSV guardado en: /Users/benjamincarbonell/Desktop/df__C_Grau_completo.csv  Filas: 175664 Columnas: 10
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Patronat
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Patronat


/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(["", " "], pd.NA)
/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(["", " "], pd.NA)
/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. 

CSV guardado en: /Users/benjamincarbonell/Desktop/df__C_Patronat_completo.csv  Filas: 172507 Columnas: 7
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Penyeta
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Castellon/C_Penyeta


/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:89: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:89: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
/var/folders/85/2m2sr7rx6g5d561zn04dcq000000gn/T/ipykernel_90127/1870222857.py:89: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], dayfirst=True, errors='coerce'

CSV guardado en: /Users/benjamincarbonell/Desktop/df__C_Penyeta_completo.csv  Filas: 175630 Columnas: 11
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Cirat
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Cirat
CSV guardado en: /Users/benjamincarbonell/Desktop/df__Cirat_completo.csv  Filas: 181992 Columnas: 11
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Onda
Aviso: no se encontró archivo para el año 2005 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 7/Onda
CSV guardado en: /Users/benjamincarbonell/Desktop/df__Onda_completo.csv  Filas: 183975 Columnas: 11
Aviso: no se encontró archivo para el año 2004 en /Users/benjamincarbonell/Desktop/TFG/Datos TFG/Transecto 8/Alcora
CSV guardado en: /Users/benjamincarbonell/Desktop/df__Alcora_completo.csv  Filas: 364728 Columnas: 6
Aviso: no se encont